In [147]:
#feed existing demands to the model and train 
#geo coding the location

In [148]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import folium
from keras.preprocessing.sequence import TimeseriesGenerator

In [149]:
data = pd.read_csv("MOCK_DATA (3).csv",index_col='Date1',parse_dates=True)
data.index.freq = 'MS'

In [150]:
df = data.drop(columns=['Time','Lattitude','Longitude'])
#df.plot(figsize=(12,6))


In [151]:
train = df.iloc[:32]
test = df.iloc[32:]


In [152]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [153]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [154]:
n_input = 3
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [155]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')     

Given the Array: 
[0.97435897 0.23076923 0.25641026]
Predict this y: 
 [[0.33333333]]


In [156]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [157]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [158]:
model.fit(generator,epochs=50)

Epoch 1/50
29/29 [==============================] - 2s 2ms/step - loss: 0.2623
Epoch 2/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1481
Epoch 3/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1156
Epoch 4/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1156
Epoch 5/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1102
Epoch 6/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1101
Epoch 7/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1104
Epoch 8/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1079
Epoch 9/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1124
Epoch 10/50
29/29 [==============================] - 0s 3ms/step - loss: 0.1041
Epoch 11/50
29/29 [==============================] - 0s 3ms/step - loss: 0.1035
Epoch 12/50
29/29 [==============================] - 0s 3ms/step - loss: 0.1049
Epoch 13/50
29/29 [==============================

In [159]:
last_train_batch = scaled_train[0:3]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))
last_train_batch


array([[[0.97435897],
        [0.23076923],
        [0.25641026]]])

In [160]:
model.predict(last_train_batch)


1/1 [==============================] - 0s 191ms/step


array([[0.3175142]], dtype=float32)

In [161]:
scaler.inverse_transform([[0.33620152]])

array([[23.11185928]])

In [162]:
map = folium.Map(location=[13.081689, 80.269032],tiles="Stamen Toner")
for i,j in data.iterrows():
    folium.Circle(
    radius=j['Pickups']*10,
    location=[j['Lattitude'],j['Longitude']],
    popup="Demand",
    color="crimson",
    fill=True,
).add_to(map)
map


In [163]:
x = data.iloc[:,2:5]


In [164]:
kmeans = KMeans(3)
kmeans.fit(x) 

C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3)

In [165]:
identified_clusters = kmeans.fit_predict(x)


C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [166]:
data_with_clusters = data.copy()
data_with_clusters['Cluster'] = identified_clusters


In [182]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="DemandPrediction")
location = geolocator.geocode("Velacherry")
print((location.latitude, location.longitude))

(12.9832366, 80.217509)


In [183]:
clust = kmeans.predict([[location.latitude,location.longitude]])
clust

C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


array([2])

In [184]:
#data1 = data_with_clusters[(data_with_clusters['Cluster'] == clust)]
xdata = df.loc[(data_with_clusters["Cluster"]==clust[0])]
xdata = xdata[:3]


In [185]:
scaled_data = scaler.transform(xdata)

In [186]:
pred_batch = scaled_data
pred_batch = pred_batch.reshape((1, n_input, n_features))
predicted = model.predict(pred_batch)

1/1 [==============================] - 0s 74ms/step


In [187]:
preds = scaler.inverse_transform(predicted)
pred_count = preds.tolist()
preas = int(pred_count[0][0])
preas


28

In [188]:
folium.Circle(
    radius= (preas)*10,
    location=[location.latitude,location.longitude],
    popup="Demand",
    color="blue",
    fill=True,
).add_to(map)
map
